### 2γ) Bayesian Networks

In [1]:
import pgmpy
import pandas as pd
import numpy as np
import os
pgmpy.__version__


'0.1.25'

In [2]:
CSV_PATH = os.path.join('harth')
csvs = os.listdir(CSV_PATH)

df6 = pd.read_csv(CSV_PATH + "/S006.csv")
df9 = pd.read_csv(CSV_PATH + "/S009.csv") 
csvs2 = csvs[:]
if "S006.csv" in csvs2:
    csvs2.pop(0)
    csvs2.pop(1)
"""csvs2.remove("S015.csv")
csvs2.remove("S021.csv")
csvs2.remove("S023.csv")"""
ultimate_df = pd.concat([df6, df9])
count = 0
for file in csvs2:
    if count >=5:
        break
    new_df = pd.read_csv(CSV_PATH + f"/{file}")
    ultimate_df = pd.concat([ultimate_df , new_df], axis=0)
    count+=1

try:
    ultimate_df.drop(columns='index', inplace=True)
except KeyError as e:
    print(e)

"['index'] not found in axis"


In [3]:
data_bayes_train = ultimate_df[['back_x', 'back_y', 'back_z', 'thigh_x', 'thigh_y', 'thigh_z', 'label']]
data_bayes_train.dropna()
data_bayes_train.drop_duplicates()
print(data_bayes_train.head())


     back_x    back_y    back_z   thigh_x   thigh_y   thigh_z  label
0 -0.760242  0.299570  0.468570 -5.092732 -0.298644  0.709439      6
1 -0.530138  0.281880  0.319987  0.900547  0.286944  0.340309      6
2 -1.170922  0.186353 -0.167010 -0.035442 -0.078423 -0.515212      6
3 -0.648772  0.016579 -0.054284 -1.554248 -0.950978 -0.221140      6
4 -0.355071 -0.051831 -0.113419 -0.547471  0.140903 -0.653782      6


Δημιουργούμε το Μοντέλο

In [4]:
from pgmpy.models import BayesianNetwork
bayesian = BayesianNetwork([('back_x', 'label'), 
                            ('back_y', 'label'), 
                            ('back_z', 'label'), 
                            ('thigh_x', 'label'),
                            ('thigh_y', 'label'),
                            ('thigh_z', 'label'), ])


In [15]:
from sklearn.model_selection import train_test_split
train_bayes, test_bayes = train_test_split(data_bayes_train, test_size=0.3, random_state=42)
sampled_data = train_bayes.sample(9, random_state=42)
sampled_test= test_bayes.sample(9, random_state=42)
sampled_test_labels = sampled_test['label']
sampled_test_no=sampled_test.drop(columns='label')

In [16]:
from pgmpy.estimators import BayesianEstimator
from pgmpy.estimators import MaximumLikelihoodEstimator
cpd_back_x =  MaximumLikelihoodEstimator(bayesian, sampled_data).estimate_cpd('back_x')
cpd_back_y =  MaximumLikelihoodEstimator(bayesian, sampled_data).estimate_cpd('back_y')
cpd_back_z =  MaximumLikelihoodEstimator(bayesian, sampled_data).estimate_cpd('back_z')
cpd_thigh_x = MaximumLikelihoodEstimator(bayesian, sampled_data).estimate_cpd('thigh_x')
cpd_thigh_y = MaximumLikelihoodEstimator(bayesian, sampled_data).estimate_cpd('thigh_y')
cpd_thigh_z = MaximumLikelihoodEstimator(bayesian, sampled_data).estimate_cpd('thigh_z')
#cpd_label = BayesianEstimator(bayesian, data_bayes_train).estimate_cpd('label')
cpd_label =   MaximumLikelihoodEstimator(bayesian, sampled_data).estimate_cpd('label')

bayesian.add_cpds(cpd_back_x,cpd_back_y,cpd_back_z,cpd_thigh_x,cpd_thigh_y,cpd_thigh_z,cpd_label)
bayesian.check_model()

True

In [17]:
from pgmpy.inference import VariableElimination

bayesian_pred = VariableElimination(bayesian)

Δοκιμη σε γνωστά δεδομένα

In [40]:
results = []
classes = {}
i = 0
for index, c in enumerate(sampled_data['label']):
    if c not in classes.values():
        classes[i] = c
        i+=1
print(classes)

for index, row in sampled_data.iterrows(): # sampled_test
    # Extract observed evidence from the current row
    evidence = {
        'back_x': row['back_x'],
        'back_y': row['back_y'],
        'back_z': row['back_z'],
        'thigh_x': row['thigh_x'],
        'thigh_y': row['thigh_y'],
        'thigh_z': row['thigh_z']
    }
    predicted_distribution = bayesian_pred.query(variables=['label'], evidence=evidence)

    # Get the most probable value for the target variable ('label')
    predicted_label = predicted_distribution.values.argmax()
    results.append(classes[predicted_label])
print(results)

{0: 3, 1: 7, 2: 13}
[3, 7, 7, 7, 7, 7, 7, 7, 13]


In [42]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(sampled_data['label'], results, average='weighted')
recall = recall_score(sampled_data['label'], results, average='weighted')
f1 = f1_score(sampled_data['label'], results, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 1.0
Recall: 1.0
F1-score: 1.0


Δοκιμή σε άγνωστα δεδομένα

In [43]:
unseen_results = []
classes = {}
i = 0
for index, c in enumerate(sampled_test['label']):
    if c not in classes.values():
        classes[i] = c
        i+=1
print(classes)


for index, row in sampled_test.iterrows(): # sampled_test
    # Extract observed evidence from the current row
    predicted_distribution = bayesian_pred.query(variables=['label'], evidence=None)

    # Get the most probable value for the target variable ('label')
    predicted_label = predicted_distribution.values.argmax()
    unseen_results.append(classes[predicted_label])
print(unseen_results)

{0: 6, 1: 7, 2: 4}
[7, 7, 7, 7, 7, 7, 7, 7, 7]


In [44]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(sampled_test['label'], unseen_results, average='weighted')
recall = recall_score(sampled_test['label'], unseen_results, average='weighted')
f1 = f1_score(sampled_test['label'], unseen_results, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.30864197530864196
Recall: 0.5555555555555556
F1-score: 0.39682539682539686


c:\codeTools\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
